<a href="https://colab.research.google.com/github/michael18781/ComputationalPhys/blob/main/Exercise1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exercise 1: The Driven Pendulum

## Preliminary code

The 2nd order ODE can be written into two first order ODEs:

with y_1 = d(theta)/dt and y_0 = theta

We have, 

d(y_0)/dt = y_1

d(y_1)/dt = -sin(y_0) - qy_1 + Fsin(2t/3).

This initial code cell imports all required modules and sets up the functions to produce the ODE solutions.

In [ ]:
import scipy.integrate as integrate
import numpy as np
import matplotlib.pyplot as plt

def pair_odes(Y, t, q, F):
  return [Y[1], -np.sin(Y[0])-q*Y[1]+F*np.sin(2*t/3)]

def main(theta_0, w_0, q, F):
  times = np.arange(0, duration, timestep)
  # Solution takes the pair of odes, initial conditions, and an array of times 
  solution = integrate.odeint(pair_odes, [theta_0, w_0], times, (q, F))

  # Extracting individual solutions
  theta = solution[:, 0]
  angular_vel = solution[:, 1]
  return times, theta, angular_vel

def convert_angle(thetas):
  # To account for 2pi errors, we make sure the angles are always within
  # the range -pi to pi
  for (i, angle) in enumerate(thetas):
    angle = (angle/(2*np.pi) - np.floor(angle/(2*np.pi)))*2*np.pi
    if angle > np.pi:
      angle = angle - 2*np.pi
    thetas[i] = angle
  return thetas

# Core task 1

We aim to solve these ODEs and plot suitable graphs to determine how accurately they agree with what we expect and energy conservation. We will also determine how the period of oscillation depends on the amplitude. Throughout this task there is no damping nor forcing.

In [ ]:
duration = 60.0
timestep = 0.01

"""
This code will plot an undamped, unforced solution of the ODEs to check it agrees with 
the theory.
"""

def plot_solution(theta_0, w_0):
  times, theta, angular_vel = main(theta_0, w_0, 0, 0)

  # Plotting the values of theta and the angular velocity over time  
  plt.plot(times, theta, label='Theta')
  plt.plot(times, angular_vel, label='Angular velocity')

  plt.legend()
  plt.ylabel('Angular displacement / velocity')
  plt.xlabel('Time / s')

plot_solution(np.pi/2, 0)

In [ ]:
duration = 60000.0
timestep = 0.01

""" 
This is some code to plot the energy of a non-damped and unforced system over
time to check that it is approximately conserved.
"""

def plot_energy(theta_0, w_0):
  times, theta, angular_vel = main(theta_0, w_0, 0, 0)

  # Plotting the scaled energy over time
  energy = 1000*(angular_vel**2 + 2*(1-np.cos(theta)))
  plt.plot(times, energy)

  plt.ylabel('Energy, up to a constant factor')
  plt.xlabel('Time / s')

plot_energy(1.56, 0)

In [ ]:
duration = 100.0
timestep = 0.01

""" 
This is some code to find the period vs amplitude for no damping and no friction. 
It will produce a plot too.
"""

def find_periods():
  # Various values of the starting position
  theta_0 = np.linspace(0, np.pi-0.01, 100)

  periods = np.zeros(len(theta_0))
  
  for (i, starting_value) in enumerate(theta_0):
    # Solution for each starting position, no damping, no forcing
    times, theta, angular_vel = main(starting_value, 0, 0, 0)

    crossing_axis = np.zeros(len(angular_vel))
    for (j, vel) in enumerate(angular_vel):
      # Checking if crossed time axis
      if np.sign(angular_vel[j-1]) == 1.0 and np.sign(vel) == -1.0:
        crossing_axis[j] = 1.0

    # Making sure we count the exact number of crossings
    crossing_axis = np.trim_zeros(crossing_axis) 
    num_times_crossed = np.sum(crossing_axis)
    # The num_times_crossed - 1 means we do not overcount the number of periods
    length_between_crossings = 0 if num_times_crossed==0 else len(crossing_axis)/(num_times_crossed-1)

    periods[i] = length_between_crossings*timestep
    

  plt.plot(theta_0, periods)
  plt.ylabel("Period / s")
  plt.xlabel("Starting value of the angle (radians)")

find_periods()

## Conclusions for core task 1

The solution for the undamped and undriven oscillator with an initial displacement theta=0.01 and zero angular velocity is:

![](https://drive.google.com/uc?export=view&id=1xvl7d6wNHi02Ycxw8K-iQ7GJDz16jwDN)

This agrees with what we expect (no changes in amplitude of oscillation, and velocity lagging behind by 1/4 of a cycle = pi/2) so we can continue to experiment now. 

The energy over time in this case is plotted over about 9500 oscillations) and stays constant to within 1.4%.

![](https://drive.google.com/uc?export=view&id=1Jgtf9VvREg-ihWdVnCzCO7KRrIdbuiFq)

Using a method of counting the time between several periods and averaging them, we deduce the period for various starting positions:

![](https://drive.google.com/uc?export=view&id=1yr2Txuee9U-90lnGUL90rESMml44do0w)

This agrees with expectation because as theta tends to zero, the period tends to 2*pi - consistent with the small angle approximation. The period for starting value pi/2 is about 7.43s - a noticeable difference from 2pi.

# Supplementary task 1

In this task we want to investigate how slightly different initial conditions can affect the long term behaviour of our solutions - the sensitivity to initial conditions.

In [ ]:
duration = 100.0
timestep = 0.01

"""
This code simply sets two solutions off at t = 0 with a very slight difference
in initial displacement. It plots the displacement vs time so we can see if 
the resulting behaviour is strongly affected by the initial displacement.
"""

def testing_sensitivity():
  # We check for two values of theta_0 = 0.2, 0.20001
  time1, theta1, angular_vel1 = main(0.2, 0, 0, 1.2)
  time2, theta2, angular_vel2 = main(0.20001, 0, 0, 1.2)

  plt.plot(time1, theta1, label='0.2')
  plt.plot(time2, theta2, label='0.20001')
  plt.xlabel('Time / s')
  plt.ylabel('Angular displacement (radians)')

testing_sensitivity()

## Conclusions for supplementary task 1

For F=1.2, we investigate whether the solutions diverge for a small difference in the starting angle. We notice that the solutions stay roughly the same until a certain point where the current state of each system is different enough to send the systems down different paths and they diverge.

![](https://drive.google.com/uc?export=view&id=1-IIbWMsq6oSHjaDhCvdOJ6Z1pBdgjRym)

# Core task 2

In this task we want to be investigating damping and driving forces. So we in turn switch on and vary the damping, and then the driving force and see the behaviour. 


In [ ]:
timestep = 0.01
duration = 100.0

"""
Code to show two plots for a few damping levels. The first plot shows displacement vs time
and the second plot shows angular velocity vs time
"""

def damping_test(theta_0, w_0):
  q = [1, 5, 10]
  f_displacement, ax = plt.subplots()
  f_velocity, ax1 = plt.subplots()
  for Q in q:
    times, theta, vel = main(theta_0, w_0, Q, 0.0)
    ax.plot(times, theta, label=f'q={Q}')
    ax1.plot(times, vel, label=f'q={Q}')

  ax.set_xlabel('Time / s')
  ax1.set_xlabel('Time / s')
  ax.set_ylabel('Angular displacement / radians')
  ax1.set_ylabel('Angular velocity / radians per s')
  ax.legend()
  ax1.legend()

damping_test(0.01, 0)

In [ ]:
timestep = 0.01
duration = 100.0

"""
Code to show the effect of a driving force on the pendulum. We have a fixed value of q = 0.5.
The F value is varied and plotted for each.
"""

def driving(theta_0, w_0):
  F_values = [0.5, 1.2, 1.44, 1.465]

  for F in F_values:
    times, theta, vel = main(theta_0, w_0, 0.5, F)
    theta = convert_angle(theta)
    f, ax = plt.subplots()
    ax.plot(times, theta)
    ax.set_title(f'q=0.5, F={F}')
    ax.set_ylabel('Angular velocity / radians per s')
    ax.set_xlabel('Time / s')

driving(0.01, 0)

## Conclusions for core task 2

Initially we just turn on some damping and check we see what is expected:

![](https://drive.google.com/uc?export=view&id=15A2hNMychyomqWB4Oqq-Wz8VClYt1pM0)

![](https://drive.google.com/uc?export=view&id=1WGn7a0OCcNlPTsK2iWCJRLhX1ch0RowA)

Then we check the plots for multiple values of F and q = 0.5. We see some awkward behaviour near plus or minus pi because of how the pendulum "goes over the top". I wrote the function convert_angle to make sure the angle stays between plus or minus pi - this is invoked after solving the ODE and corrects every value of the angle.

![](https://drive.google.com/uc?export=view&id=12UbnwcoOEJphWmI5dZ-50CHo-rysy725)

We can see that for light forcing, there is a transient solution but then we settle on the periodic forcing term.

For heavier damping we start to get some strange behaviour - some was periodic. For F = 1.2 we do not get periodic behaviour (maybe some chaos here?) but for the two higher values we can see periodic motion. For F=1.44 we appear to have period roughly equal to the forcing terms period and for F=1.465 we have a much longer period.

# Supplementary task 2

In this task we plot angular velocity against angular displacement - this is the phase space. In phase space we can see for chaos by checking if there are points which intersect with the phase space path multiple times.





In [ ]:
duration = 150.0
timestep = 0.01

"""
This code will simply plot a phase space diagram of the displacement vs velocity.
This allows us to see how the path in phase space behaves, leading to chaos.
"""

def plot_angle_vs_speed(theta_0, w_0):
  # A choice of parameters for each plot
  params = [(0.5, 0), (0.5, 0.5), (0.9, 3)]

  for pair in params:
    times, theta0, vel0 = main(theta_0, w_0, pair[0], pair[1])
    theta0 = convert_angle(theta0)
    f, ax = plt.subplots()
    ax.plot(theta0, vel0)
    ax.set_title(f'q={pair[0]}, F={pair[1]}')
    ax.set_ylabel('Angular velocity / radians per s')
    ax.set_xlabel('Angular displacement / radians')

plot_angle_vs_speed(0.01, 0)

## Conclusions for supplementary task 2

We check the plots by having no forcing term and a damping term - we expect a spiral motion:

![](https://drive.google.com/uc?export=view&id=1duzyH4ui7I_JvWhSav4AEEBPgwiY_O41)

Then we add in the forcing term and notice some normal behaviour (q = 0.5, F = 0.5) - i.e. we see the transient and then the periodic motion

![](https://drive.google.com/uc?export=view&id=1ymajXYfjyClJ8ex8-mKZVntCwHOHD92C)

Then for larger forcing values we see, for example (q = 0.9, F = 3)

![](https://drive.google.com/uc?export=view&id=1uK2H6mHfcmu6804Rq3VRsPN-YM5K2QoA)